In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 16.47 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 14.6 MB/s 
     |████████████████████████████████| 465 kB 17.0 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-10 09:32:12--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.2, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:44

In [6]:
import pandas as pd
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import RegexpTokenizer
from konlpy.tag import Mecab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd '/content/drive/MyDrive/브런치추천/'

/content/drive/MyDrive/브런치추천


In [7]:
metadata = pd.read_json('processed_data/metadata.json')

In [8]:
metadata['ID'] = metadata['magazine_id'] + metadata['article_id'] + metadata['user_id']
metadata['sentence'] = metadata['title'] + metadata['keyword_list'].apply(lambda x: ' '.join(x)) + metadata['sub_title']

In [9]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [10]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
metadata['sent_to_token'] = metadata['sentence'].apply(lambda x: custom_tokenizer(x))

In [70]:
# 임의로
metadata['topic_num'] = 0
metadata.loc[:50,'topic_num'] = 1

In [64]:
metadata['cluster'] = 0

In [41]:
from sklearn.cluster import KMeans

In [88]:
def clustering(topic_num,n_clusters):
  tmp = metadata[metadata['topic_num'] == topic_num]
  doc2vec_corpus = tmp['sentence'].tolist()
  doc_df = tmp[['ID','sent_to_token']].values.tolist()
  tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in doc_df]

  print('doc2vec을 위한 corpus 생성완료')
  print('topic이 {}인 데이터 수: {}개 '.format(topic_num,len(tagged_data)))


  model = Doc2Vec(tagged_data, vector_size=300, window=3, epochs=10, min_count=3, workers=4)

  print('Doc2vec 모델 학습 완료')

  Clustering_Method = KMeans(n_clusters=n_clusters, random_state=0)
  X = model.docvecs.vectors_docs # document vector 전체를 가져옴. 
  Clustering_Method.fit(X)

  print('클러스터링 학습 완료')

  cluster_dict = {i:[] for i in range(n_clusters)}
  for text_tags, label in zip(tagged_data, Clustering_Method.labels_):
      text, ID = text_tags
      ID = ID[0]
      metadata.loc[metadata['ID']==ID,'cluster'] = label

  print('클러스터링 결과 저장 완료')

In [89]:
clustering(1,3)

doc2vec을 위한 corpus 생성완료
topic이 1인 데이터 수 51 


Doc2vec 모델 학습 완료
클러스터 저장 완료
